https://medium.com/doma/using-nlp-bert-to-improve-ocr-accuracy-385c98ae174c

In [ ]:
import fitz
import os
import textract

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from concurrent.futures import wait
from time import sleep
from random import random

In [ ]:
import pdf2image
import shutil
import pytesseract
from pytesseract import Output, TesseractError

In [ ]:
processed_files_file = './processed_files.txt'

directories = [
            "./files/"           
            ]
save_dirs = [
            "./txt_files"
            ]

In [ ]:
import os, shutil
folder = '/path/to/folder'

def delete_folder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
            
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    
         

In [ ]:
def process_pdf(directory, save_dir, pdf_filename):
    pdf_file = os.path.join(directory, pdf_filename)
    filename = pdf_filename.replace('.pdf', '')   #no filetype
    
    with open(processed_files_file, 'r') as f:
        if pdf_file in f.read().splitlines():
            print(f'{pdf_file} has already been processed.')
            f.close()
            return
        elif(os.path.exists(os.path.join(save_dir, pdf_filename.replace('.pdf', '.txt')))):
            print(f'{pdf_file} has already been processed.')
            return
        else:    
            pdf_path = directory + pdf_filename
            img_path = './'+filename.replace(".", "_").replace(",", "_")
            
            if not os.path.exists(img_path):
                os.makedirs(img_path)
            
            pages = pdf2image.convert_from_path(pdf_path,output_folder=img_path, dpi=300, thread_count=25)
            
            try:
                for pageNum,imgBlob in enumerate(pages):
                    text = pytesseract.image_to_string(imgBlob,lang='ind', config='--psm 6')
                    with open(os.path.join(save_dir, filename+'.txt'), 'a') as the_file:
                        the_file.write(text)
                delete_folder(img_path)      
            except RuntimeError as timeout_error:
                # Tesseract processing is terminated
                pass
            
            delete_folder(img_path)      
            with open(processed_files_file, 'a') as f2:
                f2.write(pdf_file + '\n')  
                f2.close()
            sleep(random())
            return filename  


In [ ]:
def process_doc(directory, save_dir, filename):
    pdf_file = os.path.join(directory, filename)
    with open(processed_files_file, 'r') as f:
        if pdf_file in f.read().splitlines():
            print(f'{pdf_file} has already been processed.')
            f.close()
            return
        
    if filename[-4:] == ".doc":
        #return textract.process(filename).decode('utf-8').replace('\n\n','||').replace('\n','').replace('||','\n\n')
        MY_TEXT =  textract.process(os.path.join(directory, filename)).decode('utf-8').replace('\n',' ').replace('\u3000','').replace('\t','\n')
        text_file = open(os.path.join(save_dir, filename.replace('.doc', '.txt')), "w") 
        print(MY_TEXT, file=text_file)
        text_file.close()
    
    elif filename[-5:] == ".docx":
        MY_TEXT =  textract.process(os.path.join(directory, filename)).decode('utf-8').replace('\n',' ').replace('\u3000','').replace('\t','\n')

        text_file = open(os.path.join(save_dir, filename.replace('.docx', '.txt')), "w")
        print(MY_TEXT, file=text_file)
        text_file.close()

    with open(processed_files_file, 'a') as f2:
        f2.write(pdf_file + '\n')  
        f2.close()
    sleep(random())
    return filename    

In [ ]:
import os
import pytesseract
from PIL import Image
import fitz
from wand.image import Image as Image2
from concurrent.futures import ProcessPoolExecutor
from time import sleep
from random import random
import textract
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from concurrent.futures import wait

In [ ]:
directory = './files/'
save_dir = './txt_files'

pdf_filenames = [filename for filename in os.listdir(directory) if filename.endswith('.pdf')]
pdf_filenames.sort(reverse=True)
doc_filenames = [filename for filename in os.listdir(directory) if filename.endswith(tuple(['.doc','.docx']))]


with ThreadPoolExecutor(max_workers=5) as executor:
        # submit tasks and collect futures
    futures_pdf = [executor.submit(process_pdf, directory, save_dir, filename) for filename in pdf_filenames]
    for future in as_completed(futures_pdf):
            # retrieve the result
        try:
            data = future.result()
        except Exception as exc:
            print('generated an exception: %s' % (exc))
        else:
            print(data," success")
    
    
                    
    futures_doc = [executor.submit(process_doc, directory, save_dir, filename) for filename in doc_filenames]
    # process task results as they are available
    for future in as_completed(futures_doc):
        # retrieve the result
        try:
            data = future.result()
        except Exception as exc:
            print('generated an exception: %s' % (exc))
        else:
            print(data," success")            

